In [1]:
## IMPORTANT NOTE:
## I am NOT 100% sure that this is exactly what v051 uses during its test phase as the report.
## Please try running with a test file used during a training run and compare to confirm.

In [2]:
##############################################################################
##
## Script to calculate the WER (word error rate) between model output
##     and the ground truth sentences.
##
## Refer:
##     https://discourse.mozilla.org/t/standalone-wer-script/29298/4
## Code copied from here:
##     https://github.com/mozilla/DeepSpeech/blob/27444d67ec4da563aea8a42ae8daec6fe877378b/util/text.py#L85-L97 
##
##############################################################################

In [3]:
import pandas as pd

In [4]:
## All this copied from here:
## https://github.com/mozilla/DeepSpeech/blob/27444d67ec4da563aea8a42ae8daec6fe877378b/util/text.py#L85-L97
##
#
def wer(original, result):
    r"""
    The WER is defined as the editing/Levenshtein distance on word level
    divided by the amount of words in the original text.
    In case of the original having more words (N) than the result and both
    being totally different (all N words resulting in 1 edit operation each),
    the WER will always be 1 (N / N = 1).
    """
    # The WER ist calculated on word (and NOT on character) level.
    # Therefore we split the strings into words first:
    original = original.split()
    result = result.split()
    return levenshtein(original, result) / float(len(original))
##
#
def wers(originals, results):
    count = len(originals)
    rates = []
    mean = 0.0
    assert count == len(results)
    for i in range(count):
        rate = wer(originals[i], results[i])
        mean = mean + rate
        rates.append(rate)
    return rates, mean / float(count)
##
#
# The following code is from: http://hetland.org/coding/python/levenshtein.py
#
# This is a straightforward implementation of a well-known algorithm, and thus
# probably shouldn't be covered by copyright to begin with. But in case it is,
# the author (Magnus Lie Hetland) has, to the extent possible under law,
# dedicated all copyright and related and neighboring rights to this software
# to the public domain worldwide, by distributing it under the CC0 license,
# version 1.0. This software is distributed without any warranty. For more
# information, see <http://creativecommons.org/publicdomain/zero/1.0>
##
#
def levenshtein(a,b):
    "Calculates the Levenshtein distance between a and b."
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a,b = b,a
        n,m = m,n

    current = list(range(n+1))
    for i in range(1,m+1):
        previous, current = current, [i]+[0]*n
        for j in range(1,n+1):
            add, delete = previous[j]+1, current[j-1]+1
            change = previous[j-1]
            if a[j-1] != b[i-1]:
                change = change + 1
            current[j] = min(add, delete, change)

    return current[n]

In [5]:
## testing the lev dist func
#
orig = 'rohit is boy'
opstr = 'rohit is boy'
levD = levenshtein(orig.split(), opstr.split())
print(f"orig={orig}\t\topstr={opstr}\nlevD = {levD}\tmean = {levD / float(len(orig))}")
#
orig = 'rohit is boy'
opstr = 'mohit is boy'
levD = levenshtein(orig.split(), opstr.split())
print(f"orig={orig}\t\topstr={opstr}\nlevD = {levD}\tmean = {levD / float(len(orig))}")
#
orig = 'rohit is boy'
opstr = 'ro h it is toy'
levD = levenshtein(orig.split(), opstr.split())
print(f"orig={orig}\t\topstr={opstr}\nlevD = {levD}\tmean = {levD / float(len(orig))}")
#
orig = 'rohit is boy'
opstr = 'ro a it ist day'
levD = levenshtein(orig.split(), opstr.split())
print(f"orig={orig}\t\topstr={opstr}\nlevD = {levD}\tmean = {levD / float(len(orig))}")
#

orig=rohit is boy		opstr=rohit is boy
levD = 0	mean = 0.0
orig=rohit is boy		opstr=mohit is boy
levD = 1	mean = 0.08333333333333333
orig=rohit is boy		opstr=ro h it is toy
levD = 4	mean = 0.3333333333333333
orig=rohit is boy		opstr=ro a it ist day
levD = 5	mean = 0.4166666666666667


In [6]:
inFile = '/home/rohit/dpspTraining/data/modelComparisonJunkValues.csv'
dfin = pd.read_csv(inFile)
print(f"df shape = {dfin.shape}\nThe columns are:\n{[col for col in dfin.columns]}")

df shape = (3, 4)
The columns are:
['wavFileName', 'groundTruth_sent', 'baseline_NoLM_sent', 'baseline_YesLM_sent']


In [7]:
dfin

,wavFileName,groundTruth_sent,baseline_NoLM_sent,baseline_YesLM_sent
0,f1.wav,hi my name is rohit,hia mi nam is e rohiat,hi my nae is roland
1,f2.wav,i work in heidelberg,ai wuk in hi dale bag,i work in heidelburg
2,f3.wav,john is learning about a uk visa from philip,joan es lurnin abuut ai euka isa forum flip,johan is learn about I eureka is a forum flipped


In [8]:
#
## WER for Baseline No lm
rates_base_Nolm, mean_base_Nolm = wers(dfin['groundTruth_sent'].to_list() , dfin['baseline_NoLM_sent'].to_list() )
#
## WER for Baseline WITH lm
rates_base_Yeslm, mean_base_Yeslm = wers(dfin['groundTruth_sent'].to_list() , dfin['baseline_YesLM_sent'].to_list() )
#

In [9]:
print(f"mean WER : Baseline No  lm = {mean_base_Nolm}")
print(f"mean WER : Baseline Yes lm = {mean_base_Yeslm}")

mean WER : Baseline No  lm = 1.0833333333333333
mean WER : Baseline Yes lm = 0.512962962962963


In [10]:
rates_base_Nolm

[1.0, 1.25, 1.0]

In [11]:
rates_base_Yeslm

[0.4, 0.25, 0.8888888888888888]